In [29]:
import numpy as np
import os
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [30]:
data_dir = ('./data/')
class_names = ["apple", "bat", "circle", "clock", "cloud",
               "crown", "diamond", "donut", "fish",
                "hot_dog", "lightning", "mountain", "skull",
                "smiley_face", "square", "star", "sun", "t-shirt", "tree"]

len(class_names)

19

In [31]:
X = []
y = []

In [32]:
for label, class_name in enumerate(class_names):
    # Create the path to the .npy file (e.g., './data/apple.npy')
    file_path = os.path.join(data_dir, f"{class_name}.npy")

    # Load the .npy file → shape: (2000, 28, 28)
    data = np.load(file_path)

    X.append(data)

    # Add labels to y → [label, label, label, ..., label] (length = number of samples)
    y.append(np.full((data.shape[0]), label))


In [33]:
# Stack all image arrays vertically → final shape: (total_samples, 28, 28)
X = np.vstack(X)

# Stack all label arrays horizontally → final shape: (total_samples,)
y = np.hstack(y)

In [34]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

### Max Normalization

In [35]:
# # Normalize pixel values to range 0–1 (from 0–255)
# X = X.astype('float32') / 255.0
# X.shape

In [36]:
# Flatten each image from 28x28 → 784 for dense layers
# X = X.reshape(X.shape[0], -1)  # shape becomes (total_samples, 784)

In [37]:
X_train, X_, y_train, y_ = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)
# `stratify=y` ensures equal class distribution in both training and validation sets.

X_cv, X_test, y_cv, y_test = train_test_split(X_,y_, test_size=0.5, random_state=42, stratify=y_)

In [38]:
model = Sequential([
    Input(shape=(784,), name="input_vector"),
    Dense(784, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.4),
    Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.35),
    Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.3),
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.25),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.2),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.15),
    Dense(19, activation='linear')
])

In [39]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [40]:
# Train
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(patience=3, factor=0.5)
]

In [ ]:
model.fit(X_train, y_train, validation_data=(X_cv, y_cv), epochs=60, callbacks=callbacks)

Epoch 1/60
14250/14250 ━━━━━━━━━━━━━━━━━━━━ 188s 13ms/step - accuracy: 0.6539 - loss: 2.1835 - val_accuracy: 0.8001 - val_loss: 1.3548 - learning_rate: 0.0010
Epoch 2/60
14250/14250 ━━━━━━━━━━━━━━━━━━━━ 180s 13ms/step - accuracy: 0.7474 - loss: 1.5474 - val_accuracy: 0.8055 - val_loss: 1.2809 - learning_rate: 0.0010
Epoch 3/60
14250/14250 ━━━━━━━━━━━━━━━━━━━━ 175s 12ms/step - accuracy: 0.7546 - loss: 1.4857 - val_accuracy: 0.8043 - val_loss: 1.2847 - learning_rate: 0.0010
Epoch 4/60
14250/14250 ━━━━━━━━━━━━━━━━━━━━ 175s 12ms/step - accuracy: 0.7577 - loss: 1.4610 - val_accuracy: 0.8092 - val_loss: 1.2440 - learning_rate: 0.0010
Epoch 5/60
14250/14250 ━━━━━━━━━━━━━━━━━━━━ 175s 12ms/step - accuracy: 0.7588 - loss: 1.4462 - val_accuracy: 0.8151 - val_loss: 1.2269 - learning_rate: 0.0010
Epoch 6/60
14250/14250 ━━━━━━━━━━━━━━━━━━━━ 175s 12ms/step - accuracy: 0.7605 - loss: 1.4314 - val_accuracy: 0.8126 - val_loss: 1.2124 - learning_rate: 0.0010
Epoch 7/60
14250/14250 ━━━━━━━━━━━━━━━━━━━━ 17

In [ ]:
# Predict logits (raw scores)
logits = model.predict(X_cv)  # shape: (num_samples, num_classes)

# Convert logits to probabilities using softmax
y_cv_pred_probs = tf.nn.softmax(logits, axis=1).numpy()

# Show probability distribution for sample 50
print("Predicted probabilities for sample 50:", y_cv_pred_probs[50])

# Get predicted class index with the highest probability
y_cv_pred = np.argmax(y_cv_pred_probs, axis=1)

# Show predicted class index for sample 50
print("Predicted class index for sample 50:", y_cv_pred[50])

# Get class label
print("Predicted class label for sample 50:", class_names[y_cv_pred[50]])


1782/1782 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
[ 0.23491237  0.06949262  0.23087056 -0.3209829   0.25277767  0.06923839
  0.10394874 -0.04959957  0.20198688 -0.03559215 -0.19816582 -0.05125165
 -0.08544864  0.14654806 -0.08121859  0.15131322 -0.16571915  0.23785128
 -0.04661243]


In [ ]:
print(accuracy_score(y_cv,y_cv_pred))

0.865560606060606


In [ ]:
logits = model.predict(X_test)
y_test_pred_probs = tf.nn.softmax(logits, axis=1).numpy() 
y_test_pred = np.argmax(y_test_pred_probs, axis=1)
print(accuracy_score(y_test,y_test_pred))

2063/2063 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
0.8652878787878788


In [ ]:
model.save("doodle_model.keras", save_format="keras")

In [ ]:
import joblib

data_scaler = {"scaler" : scaler}

joblib.dump(data_scaler, "data_scaler.pkl")

['data_scaler.pkl']